In [13]:
import transformers
import matplotlib.pyplot as plt
import torch
import clip
from transformers.models.clip.modeling_clip import CLIPModel
from transformers.models.clip.processing_clip import CLIPProcessor
from PIL import Image

device = "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
preprocess = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [14]:

def calculate_clip_distance(image_path1, image_path2):
    # Load the images
    image1 = preprocess(Image.open(image_path1)).unsqueeze(0).to(device)
    image2 = preprocess(Image.open(image_path2)).unsqueeze(0).to(device)
    
    # Compute the image features
    with torch.no_grad():
        image_features1 = model.encode_image(image1)
        image_features2 = model.encode_image(image2)
    
    # Normalize the features
    image_features1 = image_features1 / image_features1.norm(dim=-1, keepdim=True)
    image_features2 = image_features2 / image_features2.norm(dim=-1, keepdim=True)
    
    # Calculate the distance (cosine similarity)
    distance = 1 - torch.nn.functional.cosine_similarity(image_features1, image_features2)
    
    return distance.item()